In [60]:
%pip install geopandas

In [61]:
import datapane as dp

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import geopandas as gdp

import json
from urllib.request import urlopen

import plotly.express as px

In [62]:
# Importa os dados do SIDRA
df = pd.read_csv(
    'deslocamento_municipios.txt', 
    sep='\t', 
    encoding='ISO-8859-1',
    usecols=['municipio_id', 'municipio', 'estado', 'tempo_habitual_desloc', 'qtd_pessoas', 'tempo_medio_minutos']
)

replaces = {'None': 0, '5': '005', '30': '030', '60': '060'}

df.replace(replaces, inplace=True)   

df = (
    df
    .assign(
        qtd_pessoas = df.qtd_pessoas.astype(int),
        tempo_medio_minutos = df.tempo_medio_minutos.astype(int),
        municipio_id = df.municipio_id.astype(str) 
    ) 
)
df.head()

,municipio_id,municipio,estado,tempo_habitual_desloc,qtd_pessoas,tempo_medio_minutos
0,1200013,Acrelândia,AC,005,510,0
1,1200013,Acrelândia,AC,030,1925,0
2,1200013,Acrelândia,AC,060,277,0
3,1200013,Acrelândia,AC,120,62,0
4,1200013,Acrelândia,AC,120+,28,0


In [63]:
pop_estado = (
    df 
    .groupby('estado').qtd_pessoas.sum()
    .reset_index()
    .rename(columns={'qtd_pessoas': 'total_pop'})
)
state = (
    df 
    .query('tempo_habitual_desloc != "Total"')
    .copy()
    .groupby(['estado', 'tempo_habitual_desloc']).qtd_pessoas.sum()
    .reset_index()
    .merge(pop_estado, on='estado', how='left')
)
state = (
    state 
    .assign(
        pct = round((state.qtd_pessoas / state.total_pop) * 100, 2)
    )    
)
pop_estado = px.bar(state, x='estado', y='pct', color='tempo_habitual_desloc', title='Tempo de deslocamento médio')

In [64]:
tempo_medio = (
    df 
    .query('tempo_habitual_desloc == "Total"')
    .copy()
    .groupby(['estado']).agg({'tempo_medio_minutos': 'mean', 'qtd_pessoas': 'sum'}) 
    .reset_index()
)

plot_tempo = px.bar(tempo_medio, x='estado', y='tempo_medio_minutos', title='Tempo de deslocamento médio por Estado')

In [67]:
geodf = gdp.read_file('BR_localidades\BR_Localidades_2010_v1.shp' )

test = geodf[geodf['NM_CATEGOR'] == 'CIDADE'][['CD_GEOCODM', 'NM_LOCALID', 'LONG', 'LAT', 'geometry']]

frame = df.merge(test, how='left', left_on='municipio_id', right_on='CD_GEOCODM')

In [79]:
r = dp.View(
    dp.Text("""
    # Análise de tempo médio de deslocamento por cidade - 2010
    Overview de tempo médio de deslocamento por cidades no Brasil em 2010.\n 
    Base extraída da MobiliDados, via [Base dos Dados](https://basedosdados.org/dataset/e3edf621-c491-4d74-a03a-15a759f6e638?table=01114371-3b1b-4574-a3ea-3d7d2125b4f2).
    """),
    dp.Select(
        blocks=[
            dp.Group(
                dp.Plot(pop_estado, label='Tempo habitual de deslocamento'),
                dp.Plot(plot_tempo, label='Tempo médio de deslocamento por estado'), 
                label='Plots'
            ),
            dp.Group(
                dp.DataTable(df, label='Deslocamento por cidade'),
                dp.Text("""
                ## Dicionário:\n

                - municipio_id: id do município
                - municipio: nome da cidade
                - estado: UF
                - tempo_habitual_desloc: tempo médio de deslocamento (005 = Até 5 min / 030 = Até 30 min / 060 = Até 1h / 120 = Até 2h / 120+ = Mais de 2h)""",
                label = 'Dicionário'),
                label='Tabela'
            )
            ]
            )
    )
dp.save_report(r, "mobilidade_overview.html", open=True)

App saved to ./mobilidade_overview.html